In [14]:
data_source = ['underthink-bench']

In [1]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
# Qwen/Qwen3-4B
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B
# model = LLM(model="Qwen/Qwen3-4B", gpu_memory_utilization=0.95, tensor_parallel_size=1, task="embed")





INFO 09-11 00:19:05 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-11 00:19:06 [__init__.py:239] Automatically detected platform cuda.


In [23]:
import json
data = []
with open('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/deepseek_qwen_7b_dapo_math_10k_context_linear_reward_with_summary_attention_loss_original_v2/rollout/val_140_v2.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        

In [24]:
def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1
    if right_brace_idx is None:
        retval = None
    else:
        retval = string[idx : right_brace_idx + 1]

    return retval


def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[: len(left)] == left
        assert s[-1] == "}"
        return s[len(left) : -1]
    except Exception:
        return None


def extract_boxed_answer(solution: str) -> str:
    """Extract the answer from inside a LaTeX \\boxed{} command"""
    solution = last_boxed_only_string(solution)
    solution = remove_boxed(solution)
    return solution

In [25]:
import json
with open('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/optimalThinkingBench/underthink_bench.json', 'r') as f:
    reference_dataset = json.load(f)



In [26]:
reference_dataset[0]

{'question': "A::B is a system with 4 tokens: `A#`, `#A`, `B#` and `#B`.\n\nAn A::B program is a sequence of tokens. Example:\n\n    B# A# #B #A B#\n\nTo *compute* a program, we must rewrite neighbor tokens, using the rules:\n\n    A# #A ... becomes ... nothing\n    A# #B ... becomes ... #B A#\n    B# #A ... becomes ... #A B#\n    B# #B ... becomes ... nothing\n\nIn other words, whenever two neighbor tokens have their '#' facing each-other,\nthey must be rewritten according to the corresponding rule.\n\nNow, consider the following program:\n\nB# A# B# #A #B A# A# #B A# #A B# B# #A #A A# #B #A A# #A B#\n\nReturn the final state of the program.\n",
 'answer': 'B# B#',
 'metadata': {'source_dataset': 'ab',
  'source_index': 0,
  'difficulty': {'length': 20}},
 'puzzle': 'ab',
 'uuid': '2d5bf0ea-2256-4e93-8a04-31a2a98b86df'}

In [27]:
data[0]

{'data_source': 'underthink-bench',
 'prompt': "<｜User｜>A::B is a system with 4 tokens: `A#`, `#A`, `B#` and `#B`.\n\nAn A::B program is a sequence of tokens. Example:\n\n    B# A# #B #A B#\n\nTo *compute* a program, we must rewrite neighbor tokens, using the rules:\n\n    A# #A ... becomes ... nothing\n    A# #B ... becomes ... #B A#\n    B# #A ... becomes ... #A B#\n    B# #B ... becomes ... nothing\n\nIn other words, whenever two neighbor tokens have their '#' facing each-other,\nthey must be rewritten according to the corresponding rule.\n\nNow, consider the following program:\n\nA# A# A# #A #A B# B# A# #A B# B# #A B# #A #A #A A# A# B# B#\n\nReturn the final state of the program.\n \nLet's think step by step and output the final answer within \\boxed{}.<｜Assistant｜><think>\n",
 'response': "Okay, so I have this problem where I need to compute the final state of an A::B program after applying certain rewrite rules. The program given is a long sequence of tokens: \n\nA# A# A# #A #A B

In [28]:
import reasoning_gym
from reasoning_gym.factory import create_dataset

In [29]:
puzzles = {
    "ab": {"length": 20},
    "advanced_geometry": {"min_coord": -20, "max_coord": 20},
    "knight_swap": {},
    "tsumego": {"min_board_size": 15, "max_board_size": 19, "max_stones": 20},
    "fraction_simplification": {},
    "propositional_logic": {},
    "bitwise_arithmetic": {"difficulty": 4},
    "letter_counting": {"min_words": 25, "max_words": 35},
    "maze": {"min_dist": 15, "max_dist": 25, "min_grid_size": 15, "max_grid_size": 25},
    "puzzle24": {"min_value": 8, "max_value": 10},
    "quantum_lock": {"difficulty": 25},
}

In [30]:
correctness_response_length_list = {}
from tqdm import tqdm
for d in tqdm(data):
    generated_answer = extract_boxed_answer(d['response'])
    for i in range(len(reference_dataset)):
        if reference_dataset[i]['uuid'] == d['uuid']:
            gold_entry = reference_dataset[i]
            puzzle = gold_entry['puzzle']
            metadata = gold_entry['metadata']
            dataset = reasoning_gym.create_dataset(puzzle, size=50, **puzzles[puzzle])
            break
    if gold_entry is not None:
        score = dataset.score_answer(answer=generated_answer, entry=gold_entry)
        if score < 1:
            score = 0
    if d['data_source'] not in correctness_response_length_list:
        correctness_response_length_list[d['data_source']] = []
    correctness_response_length_list[d['data_source']].append((score, len(tokenizer.encode(d['response']))))


        

100%|██████████| 2749/2749 [00:27<00:00, 99.42it/s] 


In [31]:
def compute_aucoaa(data):
    # data is a list of (correctness, response_length) tuples
    n = len(data)
    tmax = max(length for _, length in data)
    oaa_t_list = []
    for t in range(tmax + 1):
        filtered = [correct for correct, length in data if length < t]
        if filtered:
            oaa_t = sum(filtered) / n  # Denominator is always n, per definition
        else:
            oaa_t = 0.0
        oaa_t_list.append(oaa_t)
    aucoaa = sum(oaa_t_list) / len(oaa_t_list)
    return aucoaa


In [32]:
import numpy as np
for key in correctness_response_length_list.keys():
    mean_response_length = np.mean([length for _, length in correctness_response_length_list[key]])
    accuracy = np.mean([correct for correct, _ in correctness_response_length_list[key]])
    print("Data Source: ", key)
    print("Mean Response Length: ", mean_response_length)
    print("Accuracy: ", accuracy)
    print("Number of Samples: ", len(correctness_response_length_list[key]))
    print("AUC-OAA: ", compute_aucoaa(correctness_response_length_list[key]))
    print("--------------------------------")

Data Source:  underthink-bench
Mean Response Length:  5918.535103674063
Accuracy:  0.22299017824663514
Number of Samples:  2749
AUC-OAA:  0.1497214831600845
--------------------------------
